In [ ]:
%pip install langchain langchain-openai langchain-community pandas

In [ ]:
import sqlite3
import random
from datetime import datetime, timedelta

conn = sqlite3.connect("buser_dummy.db")
cursor = conn.cursor()

cursor.execute('DROP TABLE IF EXISTS bookings')
cursor.execute('DROP TABLE IF EXISTS routes')
cursor.execute('''
CREATE TABLE routes (
    route_id INTEGER PRIMARY KEY,
    origin TEXT,
    destination TEXT,
    base_price REAL
)
''')

cursor.execute('''
CREATE TABLE bookings (
    booking_id INTEGER PRIMARY KEY AUTOINCREMENT,
    route_id INTEGER,
    booking_date TEXT,
    travel_date TEXT,
    final_price REAL,
    status TEXT, -- 'confirmed', 'cancelled', 'completed'
    FOREIGN KEY(route_id) REFERENCES routes(route_id)
)
''')

routes_data = [
    (101, 'São Paulo', 'Rio de Janeiro', 85.0),
    (102, 'Belo Horizonte', 'São Paulo', 120.0),
    (103, 'Campinas', 'Curitiba', 140.0),
    (104, 'Rio de Janeiro', 'Vitória', 95.0),
    (105, 'Brasília', 'Goiânia', 60.0),
    (106, 'São Paulo', 'Florianópolis', 180.0)
]
cursor.executemany('INSERT INTO routes VALUES (?, ?, ?, ?)', routes_data)

statuses = ['completed', 'completed', 'completed', 'confirmed', 'confirmed', 'cancelled']
start_date = datetime.now() - timedelta(days=60)

bookings_batch = []
for i in range(500):
    route = random.choice(routes_data)
    r_id = route[0]
    base_price = route[3]

    # data aleatória
    days_offset = random.randint(0, 90)
    travel_dt = start_date + timedelta(days=days_offset)
    booking_dt = travel_dt - timedelta(days=random.randint(1, 10))

    # preco dinamico
    price_multiplier = random.uniform(0.8, 1.5)
    final_price = round(base_price * price_multiplier, 2)

    status = random.choice(statuses)

    bookings_batch.append((
        r_id,
        booking_dt.strftime("%Y-%m-%d"),
        travel_dt.strftime("%Y-%m-%d"),
        final_price,
        status
    ))

cursor.executemany('''
    INSERT INTO bookings (route_id, booking_date, travel_date, final_price, status)
    VALUES (?, ?, ?, ?, ?)
''', bookings_batch)

conn.commit()
print(f"bd atualizado com 500 reservas")

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent

os.environ["OPENAI_API_KEY"] = "sk-proj-"

db = SQLDatabase.from_uri("sqlite:///buser_dummy.db")

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

agent_executor = create_sql_agent(
    llm,
    db=db,
    agent_type="openai-tools",
    verbose=True
)

print("Agent pronto")

In [ ]:
queries = [
    "Qual é a rota que gerou o maior faturamento total até agora?",
    "Quantas passagens foram canceladas no total?",
    "Qual é o preço médio pago nas passagens de São Paulo para Florianópolis?",
    "Quais são as top 3 rotas com mais vendas (status 'confirmed' e 'completed')?",
    "Qual foi o faturamento total apenas no mês de Outubro de 2025? (considere a data da viagem)"
]

for q in queries:
    print(f"\n Pergunta: {q}")

    prompt_completo = f"{q} (Responda em Português do Brasil. Para valores monetários use R$. Para datas use o formato dia/mês/ano)"

    try:
        response = agent_executor.invoke(prompt_completo)
        print(f"Resposta: {response['output']}")
    except Exception as e:
        print(f"Erro: {e}")